# Script for Speciale

This script indeholde alt kode for modellerne for dette speciale t

The script indeholderfølgende:

Loading af alle packages, Importing data, Standard logistic regressione, importance Neural network Definding the two neural networks Early stopping Neural network model 1 Neural network model og XGBoost.

### Loading af alle packages 

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import animation, cm
from collections import Counter

import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from scipy import stats, optimize

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from scikeras.wrappers import KerasClassifier
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow as tf


seed_number = 1000
print_flag = 1
plot_flag = 1

In [ ]:
#pip install keras
#pip install --upgrade typing-extensions
#pip install tensorflow
#pip install --upgrade tensorflow
#pip install scikeras
#!pip install --upgrade scipy

## Importing data 

In [ ]:
# Importer de nødvendige biblioteker
import pandas as pd
from sklearn.model_selection import train_test_split

# Importer datasættene
df_ordinear = pd.read_csv('/Users/klarafaurholdt/Downloads/snapshot_table_with_active_and_churned_subscriptions.csv', low_memory=False)
df_ekstern = pd.read_csv('/Users/klarafaurholdt/Documents/Speciale/ekstern_data.csv', sep=';')

# Saml datasættene ved at foretage en venstre sammenføjning på kolonnen 'Month'
df = pd.merge(df_ordinear, df_ekstern, on='Month', how='left')

# Fjern ubrugte kolonner
useless = ["CarMake", "CarModel", "CampaignName", "PostCode", "StartDate", "ChurnDate", "ChurnPeriod", "Year"]  # Tilpas denne liste efter behov
df = df.drop(useless, axis=1)

# Opret dummies for kategoriske variabler
categorical_columns = ["ProductType", "ProductGroup", "CarType"]  # Tilpas denne liste efter behov
df = pd.get_dummies(df, columns=categorical_columns)

# Håndter NaN-værdier i kolonnen 'SumKwhHome', 
#df['SumKwhHome'].fillna(-1, inplace=True)
#df['SumKwhNetwork'].fillna(-1, inplace=True)
#df['CarType_EV'].fillna(0, inplace=True)
#df['ProductType_Clever Box'].fillna(0, inplace=True)

# Erstat kommaer med punktummer og konverter til flydende tal
df['Elpriser øst øre/kWh'] = df['Elpriser øst øre/kWh'].str.replace(',', '.').astype(float)
df['Elpriser vest øre/kWh'] = df['Elpriser vest øre/kWh'].str.replace(',', '.').astype(float)
df=df.dropna()

# Opdel data i predictors (X) og responsvariabel (y)
X = df.drop(["Churn", "Month"], axis=1)  # Juster responsvariabelnavnet efter behov
y = df["Churn"]

seed_number = 1000

# Opdel data i trænings- og testsæt
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed_number)

# Vis de første rækker af den bearbejdede DataFrame
df.head()

In [ ]:
# to get class weights we use Counter from Collections
class_weights = Counter(y)[0]/Counter(y)[1]
print('class_weights', class_weights)

#Kan gøres endnu større.

# XGBoost

## Ubalancerede XGBoost 

In [ ]:
# Definér XGBoost-modellen
model = xgb.XGBClassifier(objective="binary:logistic", random_state=seed_number, scale_pos_weight=93)

# Træn XGBoost-modellen på træningsdataene
model.fit(X_train, y_train)

# Forudsig på testdataene
y_pred = model.predict(X_test)

# Evaluér modellen
#accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy:", accuracy)

# Udskriv classification report
#print("\nClassification Report:")
#print(classification_report(y_test, y_pred))

# Udskriv forvirringsmatrice
#print("\nConfusion Matrix:")
#print(confusion_matrix(y_test, y_pred))

import numpy as np

# Forudsig sandsynligheder for begge klasser
y_pred_proba = model.predict_proba(X_test)

# Anvend tærskelværdi på sandsynlighederne for at få endelige forudsigelser
y_pred_threshold = (y_pred_proba[:, 1] >= 0.35).astype(int)

# Evaluér modellen med tærskelværdi
accuracy_threshold = accuracy_score(y_test, y_pred_threshold)
print("Accuracy with threshold 0.35:", accuracy_threshold)

# Udskriv classification report med tærskelværdi
print("\nClassification Report with threshold 0.35:")
print(classification_report(y_test, y_pred_threshold))

# Udskriv forvirringsmatrice med tærskelværdi
print("\nConfusion Matrix with threshold 0.35:")
print(confusion_matrix(y_test, y_pred_threshold))

## XGBoost balancerede data

In [1]:
filtered_df = df[df['Churn'] == 0]

# Antal datapunkter hvor Churn = 1
n_samples = len(df[df['Churn'] == 1])

# Sample fra filtered_df for at få samme antal datapunkter som Churn = 1
sampled_df = filtered_df.sample(n=n_samples)

# Find rækkerne hvor Churn = 1
churn_1_rows = df[df['Churn'] == 1]

# Saml de to datasæt
balanced_df = pd.concat([sampled_df, churn_1_rows])
balanced_df

NameError: name 'df' is not defined

In [ ]:
# Beregn gennemsnitslige elpriser pr. kWh, hvis det ikke allerede er gjort
balanced_df['AveragePrice_kWh'] = (balanced_df['Elpriser øst øre/kWh'] + balanced_df['Elpriser vest øre/kWh']) / 2

# Beregn samlede omkostninger i kroner for hjemmeforbrug og netværksforbrug
balanced_df['TotalCostHome_kroner'] = (balanced_df['SumKwhHome'] * balanced_df['AveragePrice_kWh']) / 100
balanced_df['TotalCostNetwork_kroner'] = (balanced_df['SumKwhNetwork'] * balanced_df['AveragePrice_kWh']) / 100

# Beregn forskellen i 'SumKwhHome' og 'SumKwhNetwork' mellem hver række
balanced_df['SumKwhHome_diff'] = balanced_df['SumKwhHome'].diff()
balanced_df['SumKwhNetwork_diff'] = balanced_df['SumKwhNetwork'].diff()

# Erstat eventuelle NaN-værdier i de nye kolonner med 0
balanced_df['SumKwhHome_diff'] = balanced_df['SumKwhHome_diff'].fillna(0)
balanced_df['SumKwhNetwork_diff'] = balanced_df['SumKwhNetwork_diff'].fillna(0)

# Vise det opdaterede datasæt
balanced_df

In [ ]:
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb

# Opdel data i predictors (X) og responsvariabel (y) for det filtrerede datasæt
X_filtered = balanced_df.drop(["Churn", "Month"], axis=1)
y_filtered = balanced_df["Churn"]

print('Class weights: ', Counter(y_filtered)[0] / Counter(y_filtered)[1])

# Opdel det filtrerede datasæt i trænings- og testsæt
X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = train_test_split(X_filtered,
                                                                                        y_filtered,
                                                                                        test_size=0.2,
                                                                                        random_state=seed_number)

# Beregn skaleringspositivvægt
scale_pos_weight = Counter(y_train_filtered)[0] / Counter(y_train_filtered)[1]

# Træn XGBoost-modellen på det filtrerede træningsdata
model = xgb.XGBClassifier(objective="binary:logistic",
                          random_state=seed_number,
                          scale_pos_weight=scale_pos_weight)
model.fit(X_train_filtered, y_train_filtered)

# Forudsig på det filtrerede testsæt
y_pred_filtered = model.predict(X_test_filtered)

# Evaluér modellen på det filtrerede testsæt
#accuracy_filtered = accuracy_score(y_test_filtered, y_pred_filtered)
#print("Accuracy (filtered data):", accuracy_filtered)

# Udskriv classification report
#print("\nClassification Report:")
#print(classification_report(y_test_filtered, y_pred_filtered))

# Udskriv forvirringsmatrice
#print("\nConfusion Matrix:")
#print(confusion_matrix(y_test_filtered, y_pred_filtered))

# Forudsig sandsynligheder for begge klasser
y_pred_proba = model.predict_proba(X_test_filtered)

# Anvend tærskelværdi på sandsynlighederne for at få endelige forudsigelser
y_pred_threshold = (y_pred_proba[:, 1] >= 0.35).astype(int)

# Evaluér modellen med tærskelværdi
accuracy_threshold = accuracy_score(y_test_filtered, y_pred_threshold)
print("Accuracy with threshold 0.35:", accuracy_threshold)

# Udskriv classification report med tærskelværdi
print("\nClassification Report with threshold 0.35:")
print(classification_report(y_test_filtered, y_pred_threshold))

# Udskriv forvirringsmatrice med tærskelværdi
print("\nConfusion Matrix with threshold 0.35:")
print(confusion_matrix(y_test_filtered, y_pred_threshold))

### Påvirkningsdiagram 

In [ ]:
import xgboost as xgb
from xgboost import plot_importance

model = xgb.XGBClassifier()
model.fit(X_train_filtered, y_train_filtered)

# Plot feature importance
plot_importance(model)

## XGBoost  på balancerede data Jan - Apr

In [ ]:
filtered_dfb4 = df[(df['Month'] >= 1) & (df['Month'] <= 4) & (df['Churn'] == 0)]

n_samplesb4 = len(df[(df['Month'] >= 1) & (df['Month'] <= 4) & (df['Churn'] == 1)])

sampled_dfb4 = filtered_dfb4.sample(n=n_samplesb4)

churn_1_rowsb4 = df[(df['Month'] >= 1) & (df['Month'] <= 4) & (df['Churn'] == 1)]

balanced_dfb4 = pd.concat([sampled_dfb4, churn_1_rowsb4])
balanced_dfb4

In [ ]:
# Beregn gennemsnitslige elpriser pr. kWh, hvis det ikke allerede er gjort
balanced_dfb4['AveragePrice_kWh'] = (balanced_dfb4['Elpriser øst øre/kWh'] + balanced_dfb4['Elpriser vest øre/kWh']) / 2

# Beregn samlede omkostninger i kroner for hjemmeforbrug og netværksforbrug
balanced_dfb4['TotalCostHome_kroner'] = (balanced_dfb4['SumKwhHome'] * balanced_dfb4['AveragePrice_kWh']) / 100
balanced_dfb4['TotalCostNetwork_kroner'] = (balanced_dfb4['SumKwhNetwork'] * balanced_dfb4['AveragePrice_kWh']) / 100

# Beregn forskellen i 'SumKwhHome' og 'SumKwhNetwork' mellem hver række
balanced_dfb4['SumKwhHome_diff'] = balanced_dfb4['SumKwhHome'].diff()
balanced_dfb4['SumKwhNetwork_diff'] = balanced_dfb4['SumKwhNetwork'].diff()

# Erstat eventuelle NaN-værdier i de nye kolonner med 0
balanced_dfb4['SumKwhHome_diff'] = balanced_dfb4['SumKwhHome_diff'].fillna(0)
balanced_dfb4['SumKwhNetwork_diff'] = balanced_dfb4['SumKwhNetwork_diff'].fillna(0)

# Vise det opdaterede datasæt
balanced_dfb4

In [ ]:
# Opdel data i predictors (X) og responsvariabel (y) for det filtrerede datasæt
X_filtered = balanced_dfb4.drop(["Churn", "Month"], axis=1)
y_filtered = balanced_dfb4["Churn"]

print('Class weights: ', Counter(y_filtered)[0] / Counter(y_filtered)[1])

# Opdel det filtrerede datasæt i trænings- og testsæt
X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = train_test_split(X_filtered,
                                                                                        y_filtered,
                                                                                        test_size=0.2,
                                                                                        random_state=seed_number)

# Beregn skaleringspositivvægt
scale_pos_weight = Counter(y_train_filtered)[0] / Counter(y_train_filtered)[1]

# Træn XGBoost-modellen på det filtrerede træningsdata
model = xgb.XGBClassifier(objective="binary:logistic",
                          random_state=seed_number,
                          scale_pos_weight=scale_pos_weight)
model.fit(X_train_filtered, y_train_filtered)

# Forudsig på det filtrerede testsæt
y_pred_filtered = model.predict(X_test_filtered)

# Evaluér modellen på det filtrerede testsæt
accuracy_filtered = accuracy_score(y_test_filtered, y_pred_filtered)
#print("Accuracy (filtered data):", accuracy_filtered)

# Udskriv classification report
#print("\nClassification Report:")
#print(classification_report(y_test_filtered, y_pred_filtered))

# Udskriv forvirringsmatrice
#print("\nConfusion Matrix:")
#print(confusion_matrix(y_test_filtered, y_pred_filtered))

# Forudsig sandsynligheder for begge klasser
y_pred_proba = model.predict_proba(X_test_filtered)

# Anvend tærskelværdi på sandsynlighederne for at få endelige forudsigelser
y_pred_threshold = (y_pred_proba[:, 1] >= 0.35).astype(int)

# Evaluér modellen med tærskelværdi
accuracy_threshold = accuracy_score(y_test_filtered, y_pred_threshold)
print("Accuracy with threshold 0.35:", accuracy_threshold)

# Udskriv classification report med tærskelværdi
print("\nClassification Report with threshold 0.35:")
print(classification_report(y_test_filtered, y_pred_threshold))

# Udskriv forvirringsmatrice med tærskelværdi
print("\nConfusion Matrix with threshold 0.35:")
print(confusion_matrix(y_test_filtered, y_pred_threshold))

## XGBoost på balancerede data maj-dec 

In [ ]:
filtered_dfa5 = df[(df['Month'] >= 5) & (df['Month'] <= 12) & (df['Churn'] == 0)]

n_samplesa5 = len(df[(df['Month'] >= 5) & (df['Month'] <= 12) & (df['Churn'] == 1)])

sampled_dfa5 = filtered_dfa5.sample(n=n_samplesa5)

churn_1_rowsa5 = df[(df['Month'] >= 5) & (df['Month'] <= 12) & (df['Churn'] == 1)]

balanced_dfa5 = pd.concat([sampled_dfa5, churn_1_rowsa5])
balanced_dfa5

In [ ]:
# Beregn gennemsnitslige elpriser pr. kWh, hvis det ikke allerede er gjort
balanced_dfa5['AveragePrice_kWh'] = (balanced_dfa5['Elpriser øst øre/kWh'] + balanced_dfa5['Elpriser vest øre/kWh']) / 2

# Beregn samlede omkostninger i kroner for hjemmeforbrug og netværksforbrug
balanced_dfa5['TotalCostHome_kroner'] = (balanced_dfa5['SumKwhHome'] * balanced_dfa5['AveragePrice_kWh']) / 100
balanced_dfa5['TotalCostNetwork_kroner'] = (balanced_dfa5['SumKwhNetwork'] * balanced_dfa5['AveragePrice_kWh']) / 100

# Beregn forskellen i 'SumKwhHome' og 'SumKwhNetwork' mellem hver række
balanced_dfa5['SumKwhHome_diff'] = balanced_dfa5['SumKwhHome'].diff()
balanced_dfa5['SumKwhNetwork_diff'] = balanced_dfa5['SumKwhNetwork'].diff()

# Erstat eventuelle NaN-værdier i de nye kolonner med 0
balanced_dfa5['SumKwhHome_diff'] = balanced_dfa5['SumKwhHome_diff'].fillna(0)
balanced_dfa5['SumKwhNetwork_diff'] = balanced_dfa5['SumKwhNetwork_diff'].fillna(0)

# Vise det opdaterede datasæt
balanced_dfa5

In [ ]:
# Opdel data i predictors (X) og responsvariabel (y) for det filtrerede datasæt
X_filtered = balanced_dfa5.drop(["Churn", "Month"], axis=1)
y_filtered = balanced_dfa5["Churn"]

print('Class weights: ', Counter(y_filtered)[0] / Counter(y_filtered)[1])

# Opdel det filtrerede datasæt i trænings- og testsæt
X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = train_test_split(X_filtered,
                                                                                        y_filtered,
                                                                                        test_size=0.2,
                                                                                        random_state=seed_number)

# Beregn skaleringspositivvægt
scale_pos_weight = Counter(y_train_filtered)[0] / Counter(y_train_filtered)[1]

# Træn XGBoost-modellen på det filtrerede træningsdata
model = xgb.XGBClassifier(objective="binary:logistic",
                          random_state=seed_number,
                          scale_pos_weight=scale_pos_weight)
model.fit(X_train_filtered, y_train_filtered)

# Forudsig på det filtrerede testsæt
y_pred_filtered = model.predict(X_test_filtered)

# Evaluér modellen på det filtrerede testsæt
accuracy_filtered = accuracy_score(y_test_filtered, y_pred_filtered)
#print("Accuracy (filtered data):", accuracy_filtered)

# Udskriv classification report
#print("\nClassification Report:")
#print(classification_report(y_test_filtered, y_pred_filtered))

# Udskriv forvirringsmatrice
#print("\nConfusion Matrix:")
#print(confusion_matrix(y_test_filtered, y_pred_filtered))

# Forudsig sandsynligheder for begge klasser
y_pred_proba = model.predict_proba(X_test_filtered)

# Anvend tærskelværdi på sandsynlighederne for at få endelige forudsigelser
y_pred_threshold = (y_pred_proba[:, 1] >= 0.35).astype(int)

# Evaluér modellen med tærskelværdi
accuracy_threshold = accuracy_score(y_test_filtered, y_pred_threshold)
print("Accuracy with threshold 0.35:", accuracy_threshold)

# Udskriv classification report med tærskelværdi
print("\nClassification Report with threshold 0.35:")
print(classification_report(y_test_filtered, y_pred_threshold))

# Udskriv forvirringsmatrice med tærskelværdi
print("\nConfusion Matrix with threshold 0.35:")
print(confusion_matrix(y_test_filtered, y_pred_threshold))

# Standard Logistisk Regression

In [ ]:
X_train, y_train
# drop columns with num unique = 1

# For regression models, faster convergence can be obtained by applying a standard scaling and normalization.
# This assigns equal weight to features, even if one columns has values on the order of 000's and one has small decimals.

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Counter(y_train)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt

# Creating a standard logistic regression model
LR_stand = LogisticRegression(
    class_weight = {
        0 : 1,
        1 : 50
    }
)

# Gridsearch is used to find the optimal hyperparameters
parameters = {
    'solver': ['saga'],#, 'lbfgs', 'sag' and 'newton-cg'],
    'penalty': ['none']
}
LR_stand_Grid = GridSearchCV(LR_stand, parameters, cv=5, scoring="roc_auc", n_jobs=-1)

# Training the standard logistic model
LR_stand_Grid.fit(X_train_scaled, y_train)

# Printing the optimal hyperparameters
print(LR_stand_Grid.best_estimator_)

# Model prediction
LR_stand_preds = LR_stand_Grid.predict(X_test_scaled)

# Probability predicting for ROC plot
LR_stand_probs = LR_stand_Grid.predict_proba(X_test_scaled)[:,1]

# Model prediction with threshold
LR_stand_preds = np.where(LR_stand_probs > 0.5, 1, 0)

# Creating confusion matrix
y_prediction = dict()
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = LR_stand_preds

confusion_matrix = pd.crosstab(y_prediction['status'], y_prediction["Preds"], rownames=['Actuals'], colnames=['Predicted'])
print("Confusion Matrix: \n", confusion_matrix)

# Performance measurements
performance_measurements = classification_report(LR_stand_preds, y_test ,output_dict = True)
print("Performance measurements: \n", pd.DataFrame(performance_measurements))

# Calculate ROC curve
fpr, tpr, thresholds = roc_curve(y_test, LR_stand_probs)

# Calculate AUC
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC curve (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='red', label='Baseline', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()
# Gem diagrammet som en fil
plt.savefig('ROC_curve.png', dpi=300, bbox_inches='tight')

print("Standard logistic regression is done.")

# Neuralt Netværk

In [ ]:
#Defines neural network model 1
model1 = tf.keras.Sequential()
model1.add(tf.keras.layers.Dense(8, input_dim=X_train.shape[1], activation='relu'))
model1.add(tf.keras.layers.Dense(6, activation='relu'))
model1.add(tf.keras.layers.Dense(4, activation='relu'))
model1.add(tf.keras.layers.Dense(2, activation='relu'))
model1.add(tf.keras.layers.Dense(1, activation='sigmoid'))

#Defines neural network model 2
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Dense(20, input_dim=X_train.shape[1], activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(20, activation='relu'))
model2.add(tf.keras.layers.Dropout(0.2))
model2.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [ ]:
# Compiles the two models
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# ModelCheckpoint callbacks with updated file extensions
mc1 = tf.keras.callbacks.ModelCheckpoint('model1.keras', monitor='loss', mode='min', verbose=1, save_best_only=True)
mc2 = tf.keras.callbacks.ModelCheckpoint('model2.keras', monitor='loss', mode='min', verbose=1, save_best_only=True)
  
# Transposes the y_train set
y_train = np.array(y_train)
y_train = y_train.T

# Defines early stopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)

# Make early stopping for neural network model 1
EarlyStopping_model_1 = model1.fit(X_train, y_train, epochs=100, batch_size=10, callbacks=[es, mc1], validation_split=0.2)

# Print the training loss and validation loss plot for neural network model 1
fig, ax = plt.subplots(figsize=(10, 10))
plt.plot(EarlyStopping_model_1.history['loss'], label='train')
plt.plot(EarlyStopping_model_1.history['val_loss'], label='validation error')
plt.legend()
plt.show()

# Make early stopping for neural network model 2
EarlyStopping_model_2 = model2.fit(X_train, y_train, epochs=100, batch_size=10, callbacks=[es, mc2], validation_split=0.2)

# Print the training loss and validation loss plot for neural network model 2
fig, ax = plt.subplots(figsize=(10, 10))
plt.plot(EarlyStopping_model_2.history['loss'], label='train')
plt.plot(EarlyStopping_model_2.history['val_loss'], label='validation error')
plt.legend()
plt.show()

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Define the models using a function for each
def create_NN_model1():
    NN_model1 = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Using Input layer
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(2, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    NN_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    return NN_model1

def create_NN_model2():
    NN_model2 = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Using Input layer
        tf.keras.
        layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    NN_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    return NN_model2

# Initialize and train Model 1 and Model 2
model1 = create_NN_model1()
history1 = model1.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1, validation_split=0.2)

model2 = create_NN_model2()
history2 = model2.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1, validation_split=0.2)

# Evaluate the models
def evaluate_model(model, X_test, y_test):
    probs = model.predict(X_test).flatten()
    predictions = (probs > 0.5).astype(int)
    print("Classification Report:\n", classification_report(y_test, predictions, zero_division=1))
    return probs

probs1 = evaluate_model(model1, X_test, y_test)
probs2 = evaluate_model(model2, X_test, y_test)  # Corrected function name here

# Function to plot training history including a threshold line at 0.5
def plot_training_history(history, model_name):
    epochs = range(1, len(history.history['loss']) + 1)
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history.history['loss'], label='Training Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history.history['auc'], label='Training AUC')
    plt.plot(epochs, history.history['val_auc'], label='Validation AUC')
    plt.axhline(y=0.5, color='red', linestyle='--', label='Threshold at 0.5')
    plt.title(f'Training and Validation AUC for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot the training history for both models
plot_training_history(history1, "Model 1")
plot_training_history(history2, "Model 2")

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Define the models using a function for each
def create_NN_model1():
    NN_model1 = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Using Input layer
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(2, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    NN_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    return NN_model1

def create_NN_model2():
    NN_model2 = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Using Input layer
        tf.keras.
        layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    NN_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    return NN_model2

# Initialize and train Model 1 and Model 2
model1 = create_NN_model1()
history1 = model1.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1, validation_split=0.2)

model2 = create_NN_model2()
history2 = model2.fit(X_train, y_train, epochs=50, batch_size=10, verbose=1, validation_split=0.2)

# Evaluate the models
def evaluate_model(model, X_test, y_test):
    probs = model.predict(X_test).flatten()
    predictions = (probs > 0.5).astype(int)
    print("Classification Report:\n", classification_report(y_test, predictions, zero_division=1))
    return probs

probs1 = evaluate_model(model1, X_test, y_test)
probs2 = evaluate_model(model2, X_test, y_test)  # Corrected function name here

# Function to plot training history including a threshold line at 0.5
def plot_training_history(history, model_name):
    epochs = range(1, len(history.history['loss']) + 1)
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history.history['loss'], label='Training Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history.history['auc'], label='Training AUC')
    plt.plot(epochs, history.history['val_auc'], label='Validation AUC')
    plt.axhline(y=0.5, color='red', linestyle='--', label='Threshold at 0.5')
    plt.title(f'Training and Validation AUC for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot the training history for both models
plot_training_history(history1, "Model 1")
plot_training_history(history2, "Model 2")

In [ ]:
def plot_training_history(history, model_name):
    epochs = range(1, len(history.history['loss']) + 1)
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history.history['loss'], label='Training Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Assuming the keys are known from the print statements
    auc_key = next(key for key in history.history.keys() if 'auc' in key and not 'val_' in key)
    val_auc_key = next(key for key in history.history.keys() if 'val_auc' in key)

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history.history[auc_key], label='Training AUC')
    plt.plot(epochs, history.history[val_auc_key], label='Validation AUC')
    plt.axhline(y=0.5, color='red', linestyle='--', label='Threshold at 0.5')
    plt.title(f'Training and Validation AUC for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Define the models using a function for each
def create_NN_model1():
    NN_model1 = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Using Input layer
        tf.keras.layers.Dense(8, activation='relu'),
        tf.keras.layers.Dense(6, activation='relu'),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(2, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    NN_model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    return NN_model1

def create_NN_model2():
    NN_model2 = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),  # Using Input layer
        tf.keras.
        layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    NN_model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.AUC()])
    return NN_model2

# Initialize and train Model 1 and Model 2
model1 = create_NN_model1()
history1 = model1.fit(X_train, y_train, epochs=100, batch_size=40, verbose=1, validation_split=0.2)

model2 = create_NN_model2()
history2 = model2.fit(X_train, y_train, epochs=100, batch_size=40, verbose=1, validation_split=0.2)

# Evaluate the models
def evaluate_model(model, X_test, y_test):
    probs = model.predict(X_test).flatten()
    predictions = (probs > 0.35).astype(int)
    print("Classification Report:\n", classification_report(y_test, predictions, zero_division=1))
    return probs

probs1 = evaluate_model(model1, X_test, y_test)
probs2 = evaluate_model(model2, X_test, y_test)  # Corrected function name here

# Function to plot training history including a threshold line at 0.5
def plot_training_history(history, model_name):
    epochs = range(1, len(history.history['loss']) + 1)
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history.history['loss'], label='Training Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history.history['auc'], label='Training AUC')
    plt.plot(epochs, history.history['val_auc'], label='Validation AUC')
    plt.axhline(y=0.5, color='red', linestyle='--', label='Threshold at 0.35')
    plt.title(f'Training and Validation AUC for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Plot the training history for both models
plot_training_history(history1, "Model 1")
plot_training_history(history2, "Model 2")

In [ ]:
def plot_training_history(history, model_name):
    epochs = range(1, len(history.history['loss']) + 1)
    
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history.history['loss'], label='Training Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Assuming the keys are known from the print statements
    auc_key = next(key for key in history.history.keys() if 'auc' in key and not 'val_' in key)
    val_auc_key = next(key for key in history.history.keys() if 'val_auc' in key)

    plt.subplot(1, 2, 2)
    plt.plot(epochs, history.history[auc_key], label='Training AUC')
    plt.plot(epochs, history.history[val_auc_key], label='Validation AUC')
    plt.axhline(y=0.35, color='red', linestyle='--', label='Threshold at 0.35')
    plt.title(f'Training and Validation AUC for {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

In [ ]:
plot_training_history(history1, "Model 1")
plot_training_history(history2, "Model 2")